In [ ]:
pip list
from baseline._dependencies_densenet import *
from sklearn.metrics import precision_recall_fscore_support
from sklearn.linear_model import LogisticRegression, PassiveAggressiveRegressor, LinearRegression
from sklearn.utils import shuffle

In [ ]:
#if os.path.exists("D:\\down"):
#    data_directory = "D:\\down"
#    pytorch3path = "D:\\下载\\EfficientNet-PyTorch-3D-master"
    
#else:
#    data_directory = "D:\\down"
#    pytorch3path = "D:\\下载\\EfficientNet-PyTorch-3D-master"
    
#mri_types = ("Flair_res111", "T1_res111", "T13D_res111", "T2_res111")
#SIZE = 128
#NUM_IMAGES = 64

#sys.path.append(pytorch3path)

In [ ]:
# Transform .nrrd to .nii 
# Load images
import os
from glob import glob
import numpy as np
import SimpleITK as sitk

def _load_raw_image(in_files, out_files):
    img = sitk.ReadImage(in_files)
    sitk.WriteImage(img, out_files)
    img = sitk.ReadImage(out_files)
    return img

_load_raw_image("D:\\down\\Pcnls_3\\Pcnls_baseline\\T1.nrrd",
               "D:\\down\\Pcnls_3\\Pcnls_baseline\\T1.nii.gz")
set_of_files1=(os.path.join("D:\\down\\","Pcnls_3\\Pcnls_baseline\\",item) for item in ("T1.nrrd", "T2.nrrd", "T13D.nrrd"))
set_of_files2=(os.path.join("D:\\down\\","Pcnls_3\\Pcnls_baseline\\",item) for item in ("T1.nii.gz","T2.nii.gz", "T13D.nii.gz"))
for i, j in zip(set_of_files1, set_of_files2):
    print(i, j)
    _load_raw_image(i, j)

In [ ]:
# Transform .nrrd to .nii 
# Load images
import os
from glob import glob
import numpy as np
import SimpleITK as sitk

def load_raw_image(scan_id, in_files, out_files):
    for image_file1, image_file2 in zip(in_files, out_files):
        in_file = f"D:\\down\\{scan_id}\\Pcnls_baseline\\{image_file1}"
        img = sitk.ReadImage(in_file)
        out_file = f"D:\\down\\{scan_id}\\Pcnls_baseline\\{image_file2}"
        sitk.WriteImage(img, out_file)
        #img = sitk.ReadImage(out_file)
   



In [ ]:
import copy
import tempfile
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional, Any, Mapping, Hashable
from tqdm import tqdm
import monai
from monai.config import print_config
from monai.utils import first
from monai.config import KeysCollection
from monai.data import Dataset, CacheDataset, ArrayDataset, create_test_image_3d, DataLoader
from monai.transforms import (
    Transform,
    MapTransform,
    Randomizable,
    AddChannel,
    AddChanneld,
    AsChannelFirstd,
    Compose,
    LoadImage,
    SaveImage,
    LoadImaged,
    SaveImaged,
    Lambda,
    Lambdad,
    RandSpatialCrop,
    RandSpatialCropd,
    RandSpatialCropSamplesd,
    RandCropByPosNegLabeld,
    ToTensor,
    ToTensord,
    Orientation, 
    Rotate,
    Rotated,
    Resized,
    ScaleIntensityd,
    Spacing,
    Spacingd
    
)
from monai.data import Dataset, DataLoader
from monai.networks.nets import UNet
from monai.losses import DiceLoss
from monai.inferers import SlidingWindowInferer
from monai.metrics import compute_meandice


from torchvision import models

import torch
from torch.optim import Adam
from torch.nn import Sigmoid
print_config()

In [ ]:
inputs = {
        "image": "D:\\down\\Pcnls_3\\Pcnls_baseline\\T13D_res111.nii.gz",
       # "label": "/content/maskT1.nrrd"
    }
T1 = LoadImaged(keys=["image"])(inputs)

print(f"image.shape: {T1['image'].shape}")
print(f"image.shape: {T1['image'].min()}")
print(f"image.shape: {T1['image'].max()}")
print(f"image.shape: {T1['image'].mean()}")

T2 = ScaleIntensityd(keys=["image"])(T1)

print(f"image.shape: {T2['image'].shape}")
print(f"image.shape: {T2['image'].min()}")
print(f"image.shape: {T2['image'].max()}")
print(f"image.shape: {T2['image'].mean()}")

In [ ]:
def show_plt(array, title):
    rows, cols = 1, 3
    plt.figure(figsize=(10, 10), dpi=80)
    plt.figure(1)
    plt.title(title, {"fontsize": 12,
                      "color": "black"})

    ax1 = plt.subplot(rows, cols, 1)
    ax2 = plt.subplot(rows, cols, 2)
    ax3 = plt.subplot(rows, cols, 3)

    x = array.shape[0] // 2
    y = array.shape[1] // 2
    z = array.shape[2] // 2

    plt.sca(ax1)
    plt.imshow(array[x, :, :], cmap="gray")
    plt.sca(ax2)
    plt.imshow(array[:, y, :], cmap="gray")
    plt.sca(ax3)
    plt.imshow(array[:, :, z], cmap="gray")
    
    plt.savefig(".\\test.jpg")
    
    plt.show()


In [ ]:
show_plt(T1["image"], "before resampling")


In [ ]:
T2 = AddChanneld(keys=["image"])(copy.deepcopy(T1))
T2 = Spacingd(keys=["image"], pixdim=(1,1,1))(T2)
T2 = Resized(keys=["image"], spatial_size=(256,256,128))(T2)
#T2 = ScaleIntensityd(keys=["image"])(T2)
print(f"image.shape: {T2['image'].shape}")
print(f"image.shape: {T2['image'].min()}")
print(f"image.shape: {T2['image'].max()}")
print(f"image.shape: {T2['image'].mean()}")

In [ ]:
show_plt(T2["image"][0], "after resampling")

In [ ]:
import numpy as np
import os
import nibabel as nib
import matplotlib.pyplot as plt
from nilearn.image.image import _crop_img_to as crop_img_to
from nilearn.image.image import check_niimg
from nilearn.image import new_img_like



In [ ]:
def get_foreground_from_set_of_files(scan_id, set_of_files, background_value=0, tolerance=0.00001):
    #set_of_files=["D:\\down\\Pcnls_1\\Pcnls_baseline\\Flair_res111.nii.gz",
    #              "D:\\down\\Pcnls_1\\Pcnls_baseline\\T1_res111.nii.gz",
    #              "D:\\down\\Pcnls_1\\Pcnls_baseline\\T13D_res111.nii.gz",
    #              "D:\\down\\Pcnls_1\\Pcnls_baseline\\T2_res111.nii.gz"]
    load_raw_image(scan_id, 
                   in_files=("T1_res111.nrrd", "T2_res111.nrrd", "T13D_res111.nrrd"), 
                   out_files=set_of_files)
    foreground = np.zeros((256, 256, 128), dtype=np.int8)
    for i, image_file in enumerate(set_of_files):
        dict = {"image": f"D:\\down\\{scan_id}\\Pcnls_baseline\\{image_file}"}
        #print(dict)
        image = LoadImaged(keys=["image"])(dict)
        #print(image)
        image = AddChanneld(keys=["image"])(copy.deepcopy(image))
        image = Spacingd(keys=["image"], pixdim=(1,1,1))(image)
        image = Resized(keys=["image"], spatial_size=(256,256,128))(image)
        image = ScaleIntensityd(keys=["image"])(image)
        output_dir = f"D:\\down\\{scan_id}\\Pcnls_baseline\\"
        #output_dir = os.path.join("D:\\down\\","Pcnls_1\\Pcnls_baseline\\")
        SaveImaged(keys=["image"], output_dir=output_dir)(image)
        image = image["image"][0]
        is_foreground = np.logical_or(image<(background_value-tolerance),
                                      image>(background_value+tolerance))
        #if i == 0:
            #foreground = np.zeros(is_foreground.shape, dtype=np.int8)
        foreground[is_foreground] = 1
    
    return foreground


In [ ]:
set_of_files = ("T1_res111.nii.gz", "T2_res111.nii.gz", "T13D_res111.nii.gz")
data_directory="D:\\down"
train_df = pd.read_csv(f"{data_directory}\\os.csv")
scan_ids = list(train_df["Patient"])
for scan_id in scan_ids:
    get_foreground_from_set_of_files(scan_id, set_of_files, background_value=0, tolerance=0.00001)
    
    

In [ ]:
def get_foreground_from_set_of_files_withoutsave(scan_id, set_of_files, background_value=0, tolerance=0.00001):
    #set_of_files=["D:\\down\\Pcnls_1\\Pcnls_baseline\\Flair_res111.nii.gz",
    #              "D:\\down\\Pcnls_1\\Pcnls_baseline\\T1_res111.nii.gz",
    #              "D:\\down\\Pcnls_1\\Pcnls_baseline\\T13D_res111.nii.gz",
    #              "D:\\down\\Pcnls_1\\Pcnls_baseline\\T2_res111.nii.gz"]
    load_raw_image(scan_id, 
                   in_files=("T1_res111.nrrd", "T2_res111.nrrd", "T13D_res111.nrrd"), 
                   out_files=set_of_files)
    foreground = np.zeros((256, 256, 128), dtype=np.int8)
    for i, image_file in enumerate(set_of_files):
        dict = {"image": f"D:\\down\\{scan_id}\\Pcnls_baseline\\{image_file}"}
        #print(dict)
        image = LoadImaged(keys=["image"])(dict)
        image = AddChanneld(keys=["image"])(copy.deepcopy(image))
        image = Spacingd(keys=["image"], pixdim=(1,1,1))(image)
        image = Resized(keys=["image"], spatial_size=(256,256,128))(image)
        image = ScaleIntensityd(keys=["image"])(image)
        image = image["image"][0]
        is_foreground = np.logical_or(image<(background_value-tolerance),
                                      image>(background_value+tolerance))
        #if i == 0:
            #foreground = np.zeros(is_foreground.shape, dtype=np.int8)
        foreground[is_foreground] = 1
    
    return foreground

In [ ]:
set_of_files = ("T1_res111.nii.gz", "T2_res111.nii.gz", "T13D_res111.nii.gz")
nii_foreground = get_foreground_from_set_of_files_withoutsave("Pcnls_1", set_of_files)
show_plt(nii_foreground, 'nii_foreground')


In [ ]:
def get_multi_index(foreground, rtol=1e-8):
    
    infinity_norm = max(-foreground.min(), foreground.max())
    passes_threshold = np.logical_or(foreground<-rtol*infinity_norm,
                                     foreground>rtol*infinity_norm) 
    #print(passes_threshold)
    coords = np.array(np.where(passes_threshold))
    #print(coords)
   
    start = coords.min(axis=1)
    end = coords.max(axis=1) + 1
    
    start = np.maximum(start-1, 0)
    end = np.minimum(end+1, foreground.shape[:3])
    #print(start, end)
      
    slices = [slice(s, e) for s, e in zip(start, end)]
    return slices


In [ ]:
crop = get_multi_index(nii_foreground)
print(crop)
#crop_img_to("C:\\Users\\shezi\\T2_res111\\T2_res111_trans.nii.gz", crop, copy=True)

In [ ]:
def get_crop_images_list(scan_id, set_of_files, norm_set_of_files):
#def get_crop_images(set_of_files, norm_set_of_files):
    #set_of_files=["D:\\down\\Pcnls_1\\Pcnls_baseline\\Flair_res111.nii.gz",
    #              "D:\\down\\Pcnls_1\\Pcnls_baseline\\T1_res111.nii.gz",
    #              "D:\\down\\Pcnls_1\\Pcnls_baseline\\T13D_res111.nii.gz",
    #              "D:\\down\\Pcnls_1\\Pcnls_baseline\\T2_res111.nii.gz"]
    foreground = get_foreground_from_set_of_files_withoutsave(scan_id, set_of_files)
    crop = get_multi_index(foreground)
  
    crop_images = []
    for i, image_file in enumerate(norm_set_of_files):
        file = f"D:\\down\\{scan_id}\\Pcnls_baseline\\{image_file}"
        crop_image = crop_img_to(file, crop, copy=True).get_fdata()
        #print(crop_image)
        resize_crop_image = skTrans.resize(crop_image, (128,128,128), order=1, preserve_range=True)
        norm_resize_crop_image = rescale_intensity(resize_crop_image)
        crop_images.append(norm_resize_crop_image)
    return crop_images



In [ ]:
set_of_files = ("T1_res111.nii.gz","T2_res111.nii.gz", "T13D_res111.nii.gz")
norm_set_of_files = (  
                     "T1_res111\\T1_res111_trans.nii.gz", "T2_res111\\T2_res111_trans.nii.gz", "T13D_res111\\T13D_res111_trans.nii.gz", 
                     )
nii_crop_images_list = get_crop_images_list("Pcnls_3", set_of_files, norm_set_of_files)


In [ ]:
get_norm_resize_crop_image = nii_crop_images_list[0]
print(get_norm_resize_crop_image.dtype)
print(f"image.shape: {get_norm_resize_crop_image.shape}")
print(f"image.shape: {get_norm_resize_crop_image.min()}")
print(f"image.shape: {get_norm_resize_crop_image.max()}")
print(f"image.shape: {get_norm_resize_crop_image.mean()}")
show_plt(get_norm_resize_crop_image, "norm_resize_crop_image")

In [ ]:
get_norm_resize_crop_image = nii_crop_images_list[1]
print(get_norm_resize_crop_image.dtype)
print(f"image.shape: {get_norm_resize_crop_image.shape}")
print(f"image.shape: {get_norm_resize_crop_image.min()}")
print(f"image.shape: {get_norm_resize_crop_image.max()}")
print(f"image.shape: {get_norm_resize_crop_image.mean()}")
show_plt(get_norm_resize_crop_image, "norm_resize_crop_image")

In [ ]:
get_norm_resize_crop_image = nii_crop_images_list[2]
print(get_norm_resize_crop_image.dtype)
print(f"image.shape: {get_norm_resize_crop_image.shape}")
print(f"image.shape: {get_norm_resize_crop_image.min()}")
print(f"image.shape: {get_norm_resize_crop_image.max()}")
print(f"image.shape: {get_norm_resize_crop_image.mean()}")
show_plt(get_norm_resize_crop_image, "norm_resize_crop_image")

In [ ]:
train_transforms = Compose(
    [     
        #AsChannelFirst(),
        AddChannel(),
        Rand3DElastic(#mode=("bilinear", "nearest"),
                      prob=0.25,
                      sigma_range=(5, 7),
                      magnitude_range=(50, 150),
                      spatial_size=(128, 128, 128),
                      #translate_range=(2, 2, 2),
                      #rotate_range=(np.pi/36, np.pi/36, np.pi),
                      #scale_range=(0.15, 0.15, 0.15),
                      padding_mode="zeros"),
        
         RandAffine(#mode=("bilinear", "nearest"),
                   prob=0.25,
                   spatial_size=(128, 128, 128),
                   translate_range=(0.5, 0.5, 0.5),
                   rotate_range=(np.pi/36, np.pi/36, np.pi/4),
                   scale_range=(0.15, 0.15, 0.15),
                   padding_mode="zeros"),
        #Orientation(axcodes="PLI"),
        #RandZoom(min_zoom=0.9, max_zoom=1.1, prob=0.5),
        RandSpatialCrop(roi_size=(96, 96, 96)),
       
        RandRotate(range_x=np.pi / 12, prob=0.5, keep_size=True),
        Resize(spatial_size=(128, 128, 128)),
        #RandFlip(spatial_axis=0, prob=0.5),
        ScaleIntensity(),
        EnsureType(),
    ]
)

In [ ]:
get_norm_resize_crop_image = train_transforms(nii_crop_images_list[2])
#print(get_norm_resize_crop_image.dtype)
print(f"image.shape: {get_norm_resize_crop_image.shape}")
print(f"image.shape: {get_norm_resize_crop_image.min()}")
print(f"image.shape: {get_norm_resize_crop_image.max()}")
print(f"image.shape: {get_norm_resize_crop_image.mean()}")
show_plt(get_norm_resize_crop_image[0], "norm_resize_crop_image")

In [ ]:
def load_image_3d(scan_id="Pcnls_1", data_directory="D:\\down", split="Pcnls_baseline"):
    set_of_files = ("T1_res111.nii.gz","T2_res111.nii.gz", "T13D_res111.nii.gz")
    norm_set_of_files = ("T1_res111\\T1_res111_trans.nii.gz",  "T2_res111\\T2_res111_trans.nii.gz", "T13D_res111\\T13D_res111_trans.nii.gz", 
                        )
    images_list = get_crop_images_list(scan_id, set_of_files, norm_set_of_files)
    return images_list
    #middle = len(files) // 2 
    #num_imgs2 = num_imgs // 2
    #p1 = max(0, middle - num_imgs2)
    #p2 = min(len(files), middle + num_imgs2)
    #img3d = np.stack([load_dicom_image(f) for f in files[p1:p2]]).T
    
    #if img3d.shape[-1] < num_imgs:
    #    n_zero = np.zeros((img_size, img_size, num_imgs-img3d.shape[-1]))
    #    img3d = np.concatenate((img3d, n_zero), axis=-1)
     
    #if np.min(img3d) < np.max(img3d):
    #    img3d = img3d - np.min(img3d)
    #    img3d = img3d / np.max(img3d)
    
    #return np.expand_dims(img3d, 0)
   

a = load_image_3d(scan_id="Pcnls_1")
print(a[0].shape)
#print(np.min(a), np.max(a), np.mean(a), np.median(a))

In [ ]:
torch.tensor(np.array(a))

In [ ]:
data_directory="D:\\down"
train_df = pd.read_csv(f"{data_directory}\\os.csv")
display(train_df)

df_train, df_valid = sk_model_selection.train_test_split(
    train_df,
    test_size=0.4,
    random_state=12,
    stratify=train_df["OS 1 year"],
)

dummies_train = pd.get_dummies(df_train["OS 1 year"]) # Classification
products_train = dummies_train.columns
y_train = dummies_train.values
print(y_train)
dummies_valid = pd.get_dummies(df_valid["OS 1 year"]) # Classification
products_valid = dummies_valid.columns
y_valid = dummies_valid.values

In [ ]:
df_train.head()#tail

In [ ]:
class Dataset(torch.utils.data.dataset):
    def __init__(self, paths, targets, transforms=None):
        self.paths = paths
        self.targets = targets
        self.transforms = transforms
      
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, index):
        scan_id = self.paths[index]
        if self.targets is None:
            data = load_image_3d(scan_id, data_directory="D:\\down", split=self.split)
            data = np.array(data)
            if self.transforms:
                data = self.transforms(data)
            return {"X": torch.tensor(data).float(), "id": scan_id}
        else:
            data = load_image_3d(scan_id, data_directory="D:\\down", split="Pcnls_baseline")
            data = np.array(data)
            if self.transforms:
                data = self.transforms(data)
            y = torch.tensor(self.targets[index], dtype=torch.float32)
            return {"X": torch.tensor(data).float(), "y": y}

        

In [ ]:
#class Model(nn.Module):
#    def __init__(self):
#        super().__init__()
#        self.net = EfficientNet3D.from_name("efficientnet-b0", override_params={'num_classes': 2}, in_channels=4)
#        n_features = self.net._fc.in_features
#        self.net._fc = nn.Linear(in_features=n_features, out_features=2, bias=True)
    
#    def forward(self, x):
#        out = self.net(x)
#        return out
    

In [ ]:
rows, cols = 1, 3

def show_plt_(img):
    plt.figure(figsize=(15,15), dpi=80)
    plt.figure(1)

    ax1 = plt.subplot(rows, cols, 1)
    ax2 = plt.subplot(rows, cols, 2)
    ax3 = plt.subplot(rows, cols, 3)
 
    x = img.shape[0] // 2
    y = img.shape[1] // 2
    z = img.shape[2] // 2

    plt.sca(ax1)
    plt.imshow(img[x, :, :], cmap="Spectral")
     
    plt.sca(ax2)
    plt.imshow(img[:, y, :], cmap="Spectral")
    
    plt.sca(ax3)
    plt.imshow(img[:, :, z], cmap="Spectral")
    plt.colorbar()

    plt.show()


In [ ]:
def get_last_conv_name(model):
    layer_name = None
    for name, m in model.named_modules():
        if isinstance(m, nn.Conv3d): 
            layer_name = name
        #print(m)
        #print(layer_name)
    return layer_name

#torch.cuda.empty_cache()
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#monai.networks.nets.resnet.resnet10(n_input_channels=2, n_classes=2).to(device)
#get_last_conv_name(model)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_resnet10 = monai.networks.nets.resnet.resnet10(spatial_dims=3, n_input_channels=2).to(device)
model_file = "D:\\down\\MedicalNet_pytorch_files2\\resnet_10.pth"
checkpoint = torch.load(model_file)
#model.load_state_dict(checkpoint["state_dict"], strict=False)
print(get_last_conv_name(model_resnet10))

In [ ]:
#model = monai.networks.nets.DenseNet121(spatial_dims=3, in_channels=2, out_channels=2)#.to(device)
#model = Model()
#model.to(device)
#checkpoint = torch.load(modelfile)
#model.load_state_dict(checkpoint["model_state_dict"])
#print(get_last_conv_name(model))

In [ ]:
#__all__ = ["relationship_learning", "direct_relationship_learning"]
#from sklearn.calibration import CalibratedClassifierCV
def calibrate(logits, labels):
    """
    calibrate by minimizing negative log likelihood.
    :param logits: pytorch tensor with shape of [N, N_c]
    :param labels: pytorch tensor of labels
    :return: float
    """
    scale = nn.Parameter(torch.ones(
        1, 1, dtype=torch.float32), requires_grad=True)
    optim = torch.optim.LBFGS([scale])

    def loss():
        optim.zero_grad()
        lo = torch.nn.CrossEntropyLoss()(torch.tensor(logits * scale, dtype=torch.float32), torch.tensor(labels, dtype=torch.float32))
        lo.backward()
        return lo

    state = optim.state[scale]
    for i in range(20):
        optim.step(loss)
        print(f'calibrating, {scale.item()}')
        if state['n_iter'] < optim.state_dict()['param_groups'][0]['max_iter']:
            break

    return scale.item()


def softmax_np(x):
    max_el = np.max(x, axis=1, keepdims=True)
    #print("max_el's dim{}".format(max_el.shape))
    x = x - max_el
    x = np.exp(x)
    s = np.sum(x, axis=1, keepdims=True)
    return x / s


def relationship_learning(train_logits, train_labels, validation_logits, validation_labels):
    """
    :param train_logits (ImageNet logits): [N, N_p], where N_p is the number of classes in pre-trained dataset
    :param train_labels:  [N], where 0 <= each number < N_t, and N_t is the number of target dataset
    :param validation_logits (ImageNet logits): [N, N_p]
    :param validation_labels:  [N]
    :return: [N_c, N_p] matrix  representing the conditional probability p(pre-trained class | target_class)
     """

    # convert logits to probabilities
    train_probabilities = softmax_np(train_logits * 0.8840456604957581)
    validation_probabilities = softmax_np(validation_logits * 0.8840456604957581)
  

    all_probabilities = np.concatenate((train_probabilities, validation_probabilities))
    all_labels = np.concatenate((train_labels, validation_labels))

    Cs = []
    accs = []
    classifiers = []
    for C in [1e4, 3e3, 1e3, 3e2, 1e2, 3e1, 1e1, 3.0, 1.0, 3e-1, 1e-1, 3e-2, 1e-2, 3e-3, 1e-3, 3e-4, 1e-4]:
        cls = LogisticRegression(multi_class="multinomial", C=C, fit_intercept=False)
        train_probabilities, train_labels = shuffle(train_probabilities, train_labels)
        ind_train = np.argmax(train_labels, axis=1) 
        cls.fit(train_probabilities, ind_train)
        val_predict = cls.predict(validation_probabilities)
        ind_valid = np.argmax(validation_labels, axis=1) 
        val_acc = np.sum((val_predict == ind_valid).astype(np.float)) / len(validation_labels)
        Cs.append(C)
        accs.append(val_acc)
        classifiers.append(cls)
    #cls = LinearRegression()
    #train_probabilities, train_labels = shuffle(train_probabilities, train_labels)
    #cls.fit(train_probabilities, train_labels)
    #val_predict = cls.predict(validation_probabilities)
    #val_acc = np.sum((val_predict == validation_labels).astype(np.float)) / len(validation_labels)
    #Cs.append(C)
    #accs.append(val_acc)
    #classifiers.append(cls)
    
    accs = np.asarray(accs)
    ind = int(np.argmax(accs))
    cls = classifiers[ind]
    del classifiers
    print(cls)
    print(cls.coef_.T)
    
    validation_logits = np.matmul(validation_probabilities, cls.coef_.T)
    validation_logits = torch.from_numpy(validation_logits.astype(np.float32))
    validation_labels = torch.from_numpy(validation_labels)
    #scale = calibrate(validation_logits, validation_labels)
    #print(scale.shape)
    p_target_given_pretrain = softmax_np(
        cls.coef_.T)# * scale)  # shape of [N_p, N_c], conditional probability p(target_class | pre-trained class)
   
    # in the paper, both ys marginal and yt marginal are computed
    # here we only use ys marginal to make sure p_pretrain_given_target is a valid conditional probability
    # (make sure p_pretrain_given_target[i] sums up to 1)
    pretrain_marginal = np.mean(all_probabilities, axis=0).reshape(
        (-1, 1))  # shape of [N_p, 1]
    
    p_joint_distribution = (p_target_given_pretrain * pretrain_marginal).T
    p_pretrain_given_target = p_joint_distribution / \
        np.sum(p_joint_distribution, axis=1, keepdims=True)
    return p_pretrain_given_target 


def direct_relationship_learning(train_logits, train_labels, validation_logits, validation_labels):
    """
    The direct approach of learning category relationship.
    :param train_logits (ImageNet logits): [N, N_p], where N_p is the number of classes in pre-trained dataset
    :param train_labels:  [N], where 0 <= each number < N_t, and N_t is the number of target dataset
    :param validation_logits (ImageNet logits): [N, N_p]
    :param validation_labels:  [N]
    :return: [N_c, N_p] matrix representing the conditional probability p(pre-trained class | target_class)
     """
    # convert logits to probabilities
    train_probabilities = softmax_np(train_logits * 0.8840456604957581)
    validation_probabilities = softmax_np(validation_logits * 0.8840456604957581)

    all_probabilities = np.concatenate((train_probabilities, validation_probabilities))
    all_labels = np.concatenate((train_labels, validation_labels))
    
    N_p = all_probabilities,shape[1]
    N_t = np.max(all_labels) + 1 # the number of target classes
    conditional = []
    for i in range(N_t):
        this_class = all_probabilities[all_labels == i]
        average = np.mean(this_class, axis=0, keepdims=True)
        conditional.append(average)
    print(np.concatenate(conditional).shape)
    return np.concatenate(conditional)

In [ ]:
from monai.transforms import (
    AsChannelFirst,
    Orientation,
    RandAffine,
    Rand3DElastic,
    RandRotate,
    RandFlip,
    RandZoom,  
)

torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
               
train_transforms = Compose(
    [     
        #AsChannelFirst(),
       
        Rand3DElastic(#mode=("bilinear", "nearest"),
                      prob=0.25,
                      sigma_range=(5, 7),
                      magnitude_range=(50, 150),
                      spatial_size=(128, 128, 128),
                      #translate_range=(2, 2, 2),
                      #rotate_range=(np.pi/36, np.pi/36, np.pi),
                      #scale_range=(0.15, 0.15, 0.15),
                      padding_mode="zeros"),
        
         RandAffine(#mode=("bilinear", "nearest"),
                   prob=0.25,
                   spatial_size=(128, 128, 128),
                   translate_range=(0.5, 0.5, 0.5),
                   rotate_range=(np.pi/36, np.pi/36, np.pi/4),
                   scale_range=(0.15, 0.15, 0.15),
                   padding_mode="zeros"),
        #Orientation(axcodes="PLI"),
        #RandZoom(min_zoom=0.9, max_zoom=1.1, prob=0.5),
        #RandSpatialCrop(roi_size=(96, 96, 96)),
        Resize(spatial_size=(128, 128, 128)),
        #RandRotate(range_x=np.pi / 12, prob=0.5, keep_size=True),
        #RandFlip(spatial_axis=0, prob=0.5),
        ScaleIntensity(),
        EnsureType(),
    ]
)
valid_transforms = Compose(
    [   
        #AsChannelFirst(),
        Resize(spatial_size=(128, 128, 128)),
        #RandRotate(range_x=np.pi / 12, prob=0.5, keep_size=True),
        #RandFlip(spatial_axis=0, prob=0.5),
        ScaleIntensity(),
        EnsureType(),
    ]
)

train_data_retriever = Dataset(
    paths=df_train["Patient"].values,
    targets=y_train,
    transforms=train_transforms
)
valid_data_retriever = Dataset(
    paths=df_valid["Patient"].values,
    targets=y_valid,
    transforms=valid_transforms
)
   
train_loader = DataLoader(#torch_data.DataLoader(
    train_data_retriever,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    pin_memory=torch.cuda.is_available()
    #collate_fn=pad_list_data_collate
)
valid_loader = DataLoader(#torch_data.DataLoader(
    valid_data_retriever,
    batch_size=2,
    shuffle=False,
    num_workers=0,
    pin_memory=torch.cuda.is_available()
    #collate_fn=pad_list_data_collate
)

print("done")

In [ ]:
show_plt(train_data_retriever[0]["X"][0], "train_0")
show_plt(train_data_retriever[0]["X"][1], "train_1")

In [ ]:
show_plt(valid_data_retriever[0]["X"][0], "valid_0")
show_plt(valid_data_retriever[0]["X"][1], "valid_1")

In [ ]:
class ResNet10_F(nn.Module):
    def __init__(self, pretrained=True):
        super(ResNet10_F, self).__init__()
        model_resnet10.load_state_dict(checkpoint["state_dict"], strict=False)
        self.conv1 = model_resnet10.conv1
        self.bn1 = model_resnet10.bn1
        self.relu = model_resnet10.relu
        self.maxpool = model_resnet10.maxpool
        self.layer1 = model_resnet10.layer1
        self.layer2 = model_resnet10.layer2
        self.layer3 = model_resnet10.layer3
        self.layer4 = model_resnet10.layer4
        self.avgpool = model_resnet10.avgpool
        self.__in_features = model_resnet10.fc.in_features
        self.__out_features = model_resnet10.fc.out_features
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        return x
        
    @property
    def output_dim(self):
        #print(self.__in_features)
        return self.__in_features
    
    @property
    def output_num(self):
        #print(self.__out_features)
        return self.__out_features

In [ ]:
class ResNet10_C(nn.Module):
    def __init__(self, pretrained=True):
        super(ResNet10_C, self).__init__()
        model_resnet10.load_state_dict(checkpoint["state_dict"], strict=False)
        self.fc = model_resnet10.fc

    def forward(self, x):  
        x = self.fc(x)

        return x

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.f_net = ResNet10_F(pretrained=True)
        self.c_net_1 = ResNet10_C(pretrained=True)
        
        self.c_net_2 = nn.Linear(self.f_net.output_dim, self.f_net.output_num)
        self.c_net_2.weight.data.normal_(0, 0.01)
        self.c_net_2.bias.data.fill_(0.0)
        
        self.c_net_3 = nn.Linear(self.f_net.output_dim, 3)#target
        self.c_net_3.weight.data.normal_(0, 0.01)
        self.c_net_3.bias.data.fill_(0.0)
        
        self.c_net_4 = nn.Linear(self.f_net.output_dim, 3) 
        self.c_net_4.weight.data.normal_(0, 0.01)
        self.c_net_4.bias.data.fill_(0.0)

    def forward(self, x):
        feature = self.f_net(x)
        out_1 = self.c_net_1(feature)
        out_2 = self.c_net_2(feature)
       
        out_3 = self.c_net_3(feature)#target
        out_4 = self.c_net_4(feature)

        return out_1, out_2, out_3, out_4
  

In [ ]:
#net = Net().cuda()
model = Net()#.cuda()
model.to(device)
print(get_last_conv_name(model))
#batch_size = 2
log_var_a = torch.zeros((400,), requires_grad=True)
log_var_b = torch.zeros((3,), requires_grad=True)
#print(get_last_conv_name(model))
a = torch.exp(-log_var_a) ** 2
b = torch.exp(-log_var_b) ** 2

In [ ]:
def get_feature(loader):
    idx = 0
    train_labels_list = []
    imagenet_labels_list = []
    for step, batch in enumerate(loader, 1):
        model.eval()
        train_inputs = batch["X"] 
        train_labels = batch["y"] 
        #for train_inputs, train_labels in tqdm(loader):
        train_labels_list.append(train_labels)
        train_inputs, train_labels = train_inputs.to(device), train_labels.to(device)
        imagenet_labels, var_1, _, var_2 = model(train_inputs)
        #print(imagenet_labels)
        imagenet_labels = imagenet_labels.detach().cpu().numpy()
        imagenet_labels_list.append(imagenet_labels)
       
    all_train_labels = np.concatenate(train_labels_list, 0)
    all_imagenet_labels = np.concatenate(imagenet_labels_list, 0)

    return all_imagenet_labels, all_train_labels

train_imagenet_labels, train_train_labels = get_feature(train_loader)
val_imagenet_labels, val_train_labels = get_feature(valid_loader)
relationship = relationship_learning(train_imagenet_labels, train_train_labels, val_imagenet_labels, val_train_labels)
print("done")

In [ ]:
import torch.nn.functional as F
torch.backends.cudnn.benchmark = True
class Trainer:
    def __init__(
        self, 
        model,
        device,
        optimizer, 
        scheduler,
        criterion1,
        criterion2,
        trade_off,
        log_var_a,
        log_var_b,
        a,
        b
    ):
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.criterion1 = criterion1
        self.criterion2 = criterion2
        self.trade_off = trade_off
        self.log_var_a = log_var_a
        self.log_var_b = log_var_b
        self.a = a
        self.b = b
        
        self.best_valid_score = np.inf
        self.n_patience = 0
        self.lastmodel = None
        
    def fit(self, epochs, train_loader, valid_loader, save_path, patience):
        writer = SummaryWriter()
        start_time = time.time()
        for n_epoch in range(1, epochs + 1):
            self.info_message("EPOCH: {}", n_epoch)
            
            train_ce_loss, train_imagenet_loss, train_loss, train_time = self.train_epoch(train_loader)
            
            valid_ce_loss, valid_imagenet_loss, valid_loss, valid_auc, valid_time = self.valid_epoch(valid_loader)
            
            
            
            writer.add_scalar("Train_ce_loss", train_ce_loss, n_epoch)
            writer.add_scalar("Train_imagenet_loss", train_imagenet_loss, n_epoch)
            
            self.info_message(
                "[Epoch Train: {}] loss: {:.4f}, time: {:.2f} s",
                n_epoch, train_loss, train_time
            )
            writer.add_scalar("Train_loss", train_loss, n_epoch)
            
            
            
            writer.add_scalar("Valid_ce_loss", valid_ce_loss, n_epoch)
            writer.add_scalar("Valid_imagenet_loss", valid_imagenet_loss, n_epoch)
            
            self.info_message(
                "[Epoch Valid: {}] loss: {:.4f}, auc: {:.4f}, time: {:.2f} s ",
                n_epoch, valid_loss, valid_auc, valid_time
            )
            writer.add_scalar("Valid_auc", valid_auc, n_epoch)
            
            if self.best_valid_score > valid_loss:
                self.save_model(n_epoch, save_path)
                self.info_message(
                    "Auc improved from {:.4f} to {:.4f}. Saved model to '{}'",
                    self.best_valid_score, valid_loss, self.lastmodel
                )
                self.best_valid_score = valid_loss
                self.n_patience = 0
            else:
                self.n_patience += 1
            writer.add_scalar("Valid_loss", valid_loss, n_epoch)
        
            if self.n_patience >= patience:
                self.info_message("\nValid auc didn't improve last {} epochs.", patience)
                break
        time_took = time.time() - start_time
        print(f"train completed, time took: {hms_string(time_took)}.")
        writer.close()
                
                
    def train_epoch(self, train_loader):
        self.model.train()
        t = time.time()
        sum_ce_loss = 0
        sum_imagenet_loss = 0
        sum_loss = 0
    
        for step, batch in enumerate(train_loader, 1):
            train_inputs = batch["X"].to(self.device)
            train_labels = batch["y"].to(self.device)
            ind = train_labels.argmax(dim=1)
            ind = torch.Tensor.cpu(ind)
            #print(ind)
            #print(relationship[ind].shape)
            imagenet_targets = torch.from_numpy(relationship[ind]).to(self.device).float()
            imagenet_outputs, var_1, train_outputs, var_2 = self.model(train_inputs)
            #print(var)
            #outputs = self.model(X).squeeze(1)
            #loss = self.criterion(outputs, targets)
            #sum_loss += loss.item()
            #train_outputs = train_outputs.squeeze(1)
            
            # assume that each logit value is drawn from Gaussian distribution, therefore the whole logit vector is drawn from multi-dimensional Gaussian distribution
            epsilon_1 = torch.randn(var_1.size()).to(self.device)
            logit_1 = imagenet_outputs + torch.mul(var_1, epsilon_1)
            #print(logit)
            imagenet_outputs = logit_1#F.softmax(logit, dim=1)
            
            epsilon_2 = torch.randn(var_2.size()).to(self.device)
            logit_2 = train_outputs + torch.mul(var_2, epsilon_2)
            #print(logit)
            train_outputs = logit_2#F.softmax(logit, dim=1)
            
            #print(self.log_var_a.shape)
            #print(self.log_var_a)
            self.log_var_a = torch.tensor(var_1, requires_grad=True)
            self.log_var_b = torch.tensor(var_2, requires_grad=True)
            #print(self.log_var_a.shape)
            #print(self.log_var_a)
            
            self.a = torch.exp(-self.log_var_a) ** 2
            self.b = torch.exp(-self.log_var_b) ** 2

            #print(F.softmax(logit, dim=1))
            #print(train_outputs)
            #ce_loss = self.criterion1(train_outputs, train_labels)
            #imagenet_loss = - imagenet_targets * self.criterion2(imagenet_outputs)
            #imagenet_loss = torch.mean(torch.sum(imagenet_loss, dim=-1))
            #loss = ce_loss + self.trade_off * imagenet_loss
            imagenet_loss = - self.a * imagenet_targets * self.criterion2(imagenet_outputs)
            imagenet_loss = torch.mean(torch.sum(imagenet_loss, dim=-1))
            
            ce_loss = - self.b * train_labels * self.criterion2(train_outputs)
            ce_loss = torch.mean(torch.sum(ce_loss, dim=-1))
            #ce_loss = self.criterion1(train_outputs, train_labels)
            
            #print(imagenet_targets.shape)
            #print(imagenet_outputs.shape)
            #print(imagenet_loss)
            #print(ce_loss)
            #print(ce_loss.item())
            #print(imagenet_loss)
            #print(imagenet_loss.item())
            #print(self.trade_off)
            loss = ce_loss + imagenet_loss + torch.mean(self.log_var_a) + torch.mean(self.log_var_b)
            
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
            sum_ce_loss += ce_loss.item()
            sum_imagenet_loss += imagenet_loss.item()
            sum_loss += loss
            
            message = "Train Step {}/{}, train_ce_loss: {:.4f}"
            self.info_message(message, step, len(train_loader), sum_ce_loss/step, end="\r")
            
            message = "Train Step {}/{}, train_imagenet_loss: {:.4f}"
            self.info_message(message, step, len(train_loader), sum_imagenet_loss/step, end="\r")
            
            message = "Train Step {}/{}, train_loss: {:.4f}"
            self.info_message(message, step, len(train_loader), sum_loss/step, end="\r")
        self.scheduler.step()
        return sum_ce_loss/len(train_loader), sum_imagenet_loss/len(train_loader), sum_loss/len(train_loader), int(time.time()-t)

    
    def valid_epoch(self, valid_loader):
        self.model.eval()
        t = time.time()
        sum_ce_loss = 0
        sum_imagenet_loss = 0
        sum_loss = 0
        y_all = []
        outputs_all = []
        num_correct = 0.0
        metric_count = 0
        self.model.eval()
        with torch.no_grad():
            for step, batch in enumerate(valid_loader, 1):
                val_inputs = batch["X"].to(self.device)
                #print(X.shape)
                val_labels = batch["y"].to(self.device)
                #print(targets.shape)
                #outputs = self.model(X).squeeze(1)
                ind = val_labels.argmax(dim=1)
                ind = torch.Tensor.cpu(ind)
                #print(ind)
                #print(relationship[ind].shape)
                val_imagenet_targets = torch.from_numpy(relationship[ind]).to(self.device).float()
                val_imagenet_outputs, var1_, val_outputs, var_2 = self.model(val_inputs)
                #val_outputs = val_outputs.squeeze(1)
                #loss = self.criterion(outputs, targets)
                #sum_loss += loss.item()

                #ce_loss = self.criterion1(val_outputs, val_labels)
                #sum_ce_loss += ce_loss.item()
            
                #imagenet_loss = - val_imagenet_targets * self.criterion2(val_imagenet_outputs)
                #imagenet_loss = torch.mean(torch.sum(imagenet_loss, dim=-1))
                #sum_imagenet_loss += imagenet_loss.item()
            
                #loss = ce_loss + self.trade_off * imagenet_loss
                #sum_loss += loss
                imagenet_loss = - self.a * val_imagenet_targets * self.criterion2(val_imagenet_outputs)
                imagenet_loss = torch.mean(torch.sum(imagenet_loss, dim=-1))
                ce_loss = - self.b * val_labels * self.criterion2(val_outputs)
                ce_loss = torch.mean(torch.sum(ce_loss, dim=-1))
                # ce_loss = self.criterion1(train_outputs, train_labels)
                
                # print(imagenet_targets.shape)
                # print(imagenet_outputs.shape)
                # print(imagenet_loss)
                # print(ce_loss)
                # print(ce_loss.item())
                # print(imagenet_loss)
                # print(imagenet_loss.item())
                # print(self.trade_off)
                loss = ce_loss + imagenet_loss + torch.mean(self.log_var_a) + torch.mean(self.log_var_b)

                sum_ce_loss += ce_loss.item()
                sum_imagenet_loss += imagenet_loss.item()
                sum_loss += loss.item()
                
                value = torch.eq(torch.sigmoid(val_outputs).argmax(dim=1), val_labels.argmax(dim=1))
                metric_count += len(value)
                num_correct += value.sum().item()
        
                y_all.extend(batch["y"].argmax(dim=1).tolist())
                outputs_all.extend(torch.sigmoid(val_outputs).argmax(dim=1).tolist())
            
                message = "Valid Step {}/{}, valid_ce_loss: {:.4f}"
                self.info_message(message, step, len(valid_loader), sum_ce_loss/step, end="\r")
                message = "Valid Step {}/{}, valid_imagenet_loss: {:.4f}"
                self.info_message(message, step, len(valid_loader), sum_imagenet_loss/step, end="\r")
                message = "Valid Step {}/{}, valid_loss: {:.4f}"
                self.info_message(message, step, len(valid_loader), sum_loss/step, end="\r")
        
        
            auc = num_correct / metric_count
            #metric_values.append(metric)
            #auc = roc_auc_score(y_all, torch.Tensor.cpu(outputs))
            #outputs_all = [1 if y > 0.5 else 0 for y in outputs_all]
            #auc_ = roc_auc_score(y_all, outputs_all)
            print(precision_recall_fscore_support(y_all, outputs_all))
        
            return sum_ce_loss/len(valid_loader), sum_imagenet_loss/len(valid_loader), sum_loss/len(valid_loader), auc, int(time.time()-t) 
        
    
    def save_model(self, n_epoch, save_path):
        self.lastmodel = f"{save_path}-cofinetuneresnet_min_loss_model_classification3d_array_newloss.pth"
        torch.save(
            {
                "model_state_dict": self.model.state_dict(),
                "optimizer_state_dict": self.optimizer.state_dict(),
                "best_valid_score": self.best_valid_score,
                "n_epoch": n_epoch,
            },
            self.lastmodel,
        )

    
    @staticmethod
    def info_message(message, *args, end="\n"):
        print(message.format(*args), end=end)


In [ ]:
#torch.cuda.empty_cache()
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_all_type(train_loader, valid_loader, mri_type="all"):
    model = Net()#.cuda()
    model.to(device)
    optimizer = torch.optim.Adam([p for p in model.parameters()] + [log_var_a] + [log_var_b], lr=1e-3)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    criterion1 = torch.nn.CrossEntropyLoss()#torch_functional.binary_cross_entropy_with_logits
    criterion2 = torch.nn.LogSoftmax(dim=-1)
    trade_off = 2.3
    
    trainer = Trainer(
        model,
        device,
        optimizer,
        scheduler,
        criterion1, 
        criterion2,
        trade_off,
        log_var_a,
        log_var_b,
        a,
        b
    )
    
    history = trainer.fit(
        20, 
        train_loader, 
        valid_loader, 
        f"{mri_type}",
        10,
    )
    
    return trainer.lastmodel

if __name__ == "__main__":
    modelfile = train_all_type(train_loader, valid_loader, "all")
    print(modelfile)

In [ ]:
a = load_image_3d(scan_id="Pcnls_84")

a = np.array(a)
a = torch.tensor(a, dtype=torch.float32)
print(a.shape)
print(a[0].shape)
show_plt(a[0], "Flair")
b = a.unsqueeze(0)
print(b.shape)

In [ ]:
def get_last_conv_name_(model):
    layer_name = None
    mm = None
    for name, m in model.named_modules():
        if isinstance(m, nn.Conv3d):
            layer_name = name
            mm = m
            #print(m)
            #print(layer_name)
    return layer_name

model = Net()
print(get_last_conv_name_(model))

In [ ]:
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelfile = "all-cofinetuneresnet_min_loss_model_classification3d_array_newloss.pth"
checkpoint_new = torch.load(modelfile)
class Net_New(nn.Module):
    def __init__(self):
        super(Net_New, self).__init__()
        net = Net()
        net.load_state_dict(checkpoint_new["model_state_dict"]) 
        self.net_1 = net.f_net 
        self.net_3 = net.c_net_3
    def forward(self, x):
        x = self.net_1(x)
        out_3 = self.net_3(x)#target
        return out_3

print(get_last_conv_name_(Net_New()))        

In [ ]:
def medCAM(channelimg):
    for i, j in enumerate(("Flair", #"T1", "T13D", 
                           "T2")):
        #print(j)
        channelimg = np.array(channelimg)
        tensor = torch.tensor(channelimg, dtype=torch.float32).unsqueeze(0)
   
        tensor = tensor.to(device)#cuda()
        torch.cuda.empty_cache()
        #model = monai.networks.nets.DenseNet121(spatial_dims=3, in_channels=2, out_channels=2)#.to(device)
        #model = Model()
        
        Model = Net_New()#.cuda()
        Model.to(device)
        #checkpoint = torch.load(modelfile)
        #Model.load_state_dict(checkpoint["model_state_dict"], strict=False)
        layer_name = get_last_conv_name_(Model)
        #print(layer_name)
        model = medcam.inject(Model, output_dir="attention_maps", backend="gcam", layer=layer_name, label=2, replace=True, return_score=True)
        model.eval()
        output = model(tensor)
        
        xyz = medcam.save(output[0].squeeze(0).squeeze(0), f"attention_maps\\{j}\\{modelfile}", heatmap=True, raw_input=None)
        
        x = torch.Tensor.cpu(output[0].squeeze(0).squeeze(0))
        y = channelimg[i]
        z = x + y
    
        xyz_ = medcam.save(z, f"attention_maps_\\{j}\\{modelfile}", heatmap=True, raw_input=None)
        
        #show_plt(z)
        show_plt_(z)
        #show_plt_(torch.Tensor.cpu(output[0].squeeze(0).squeeze(0)))

In [ ]:
medCAM(a)

In [ ]:
set_of_files = ("Flair_res111.nii.gz", "T1_res111.nii.gz", "T13D_res111.nii.gz", "T2_res111.nii.gz")
data_directory="D:\\down"
df_test = pd.read_csv(f"{data_directory}\\test.csv")
display(df_test)
scan_ids_test = list(df_test["Patient"])
for scan_id_test in scan_ids_test:
    get_foreground_from_set_of_files(scan_id_test, set_of_files, background_value=0, tolerance=0.00001)

In [ ]:
dummies_test = pd.get_dummies(df_test["OS 1 year"]) # Classification
cats = [0,1,2]
dummies_test = dummies_test.T.reindex(cats).T.fillna(0)
products_test = dummies_test.columns
y_test = dummies_test.values
print(products_test)
print(y_test)


In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix

# Plot a confusion matrix.
# cm is the confusion matrix, names are the names of the classes.
def plot_confusion_matrix(cm, names, title="Confusion Matrix", cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation="nearest", cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(names))
    plt.xticks(tick_marks, names)#, rotation=45)
    plt.yticks(tick_marks, names)
    plt.tight_layout()
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    

# Plot an ROC. pred - the predictions, y - the expected output.
def plot_roc(pred,y):
    fpr, tpr, _ = roc_curve(y, pred, pos_label={1})
    roc_auc = auc(fpr, tpr)
    #print(roc_auc)
    plt.figure()
    plt.plot(fpr, tpr, label="ROC Curve (Area = %0.2f)" % roc_auc)
    plt.plot([0, 1], [0, 1], "k--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Receiver Operating Characteristic (ROC)")
    plt.legend(loc="lower right")
    plt.savefig(".\\roc.jpg")
    plt.show()

In [ ]:
def predict(products_test):
    test_transforms = Compose(
        [
            #AsChannelFirst(),
            Resize(spatial_size=(128, 128, 128)),
            #RandRotate(range_x=np.pi / 12, prob=0.5, keep_size=True),
            #RandFlip(spatial_axis=0, prob=0.5),
            ScaleIntensity(),
            EnsureType(),
        ]
    )
    
    test_data_retriever = Dataset(
        paths=df_test["Patient"].values,
        targets=y_test,
        transforms=test_transforms
    )
    
    test_loader = DataLoader(#torch_data.DataLoader(
        test_data_retriever,
        batch_size=1,
        shuffle=False,
        num_workers=0,
        pin_memory=torch.cuda.is_available()
        #collate_fn=pad_list_data_collate
    )

    Model = Net_New()#.cuda()
    Model.to(device)
    
    
    Model.eval()
    
    
    y_label = []
    y_pred = []
    num_correct = 0.0
    metric_count = 0
    
    with torch.no_grad():
        for e, batch in enumerate(test_loader,1):
            print("{}/{}".format(e, len(test_loader)), end="\r")
            test_inputs = batch["X"].to(device)
            test_labels = batch["y"].to(device)
            ind = test_labels.argmax(dim=1)
            ind = torch.Tensor.cpu(ind)
            test_outputs = Model(test_inputs)
           
            value = torch.eq(torch.sigmoid(test_outputs).argmax(dim=1), test_labels.argmax(dim=1))
            metric_count += len(value)
            num_correct += value.sum().item()
            
            y_label.append(batch["y"].argmax(dim=1).tolist())
            y_pred.append(torch.sigmoid(test_outputs).argmax(dim=1).tolist())
        
        auc = num_correct / metric_count
        precision, recall, fscore, support = precision_recall_fscore_support(y_label, y_pred, average='macro')
        plot_roc(y_label, y_pred) 
        cm = confusion_matrix(y_label, y_pred)
        #np.set_printoptions(precision=2)
        cm_normalized = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]

        plt.figure()
        plot_confusion_matrix(cm_normalized, products_test, title="Normalized Confusion Matrix")
        #plot_confusion_matrix(cm, products_test, title='Normalized confusion matrix')
        plt.savefig(".\\cm.jpg")
        plt.show()
        
        print("auc =", auc)
        print("precision =", precision)
        print("recall =", recall)
        print("fscore =", fscore)
        return auc, precision, recall, fscore, support
        
          






In [ ]:
pred = predict(products_test)

In [ ]:
print(pred)